In [0]:
%sql
CREATE OR REPLACE FUNCTION clean_with_null(column_value string) RETURNS STRING

RETURN CASE
  WHEN column_value IS NULL THEN NULL
  WHEN regexp_replace(column_value, '[€%\\p{Z}]', '') IN ('', 'null', 'NaN') --First, remove every space/separator in Unicode, in addition to % and €
    OR column_value RLIKE '^[€%,.\\p{Z}]+$' THEN NULL --It consists of '[€%,.\\p{Z}]' and nothing else
	
  ELSE column_value
END;

In [0]:
%sql
CREATE OR REPLACE FUNCTION clean_for_decimal_type(column_value string) RETURNS DECIMAL(29,2)

RETURN CASE 
    WHEN column_value IS NULL THEN NULL
    WHEN regexp_replace(column_value, '[€%\\p{Z}]', '') IN ('', 'null', 'NaN') --First, remove every space/separator in Unicode, in addition to % and €
      OR column_value RLIKE '^[€%,.\\p{Z}]+$' THEN NULL --It consists of '[€%,.\\p{Z}]' and nothing else
    
    ELSE CAST(regexp_replace(regexp_replace(regexp_replace(column_value, '[€%\\p{Z}]', ''),
                                                                         '[,.](?=.{3,})', ''),--Replaces any comma/period (with at least 3 characters afterwards) with empty string
                                                                         ',([0-9]{2})$', '.\1'--Replaces the matched comma (before the 2 digits at the end) with a period.
             ) AS DECIMAL(29,2))
END;


In [0]:
%sql
CREATE OR REPLACE FUNCTION clean_for_date_type(column_value string) RETURNS DATE

RETURN CASE
  WHEN column_value IS NULL THEN NULL
  WHEN regexp_replace(column_value, '[€%\\p{Z}]', '') IN ('', 'null', 'NaN') --First, remove every space/separator in Unicode, in addition to % and €
    OR column_value RLIKE '^[€%,.\\p{Z}]+$' THEN NULL --It consists of '[€%,.\\p{Z}]' and nothing else
  
  --'d' accepts 1 or 2 digits for days
  WHEN column_value RLIKE '-Jan-' THEN TO_DATE(column_value, 'd-MMM-yyyy')
  --If it's already "Fev", REPLACE does nothing
  WHEN column_value RLIKE '-F(eb|ev)-' THEN TO_DATE(REPLACE(column_value, 'Fev', 'Feb'), 'd-MMM-yyyy')
  WHEN column_value RLIKE '-Mar-' THEN TO_DATE(column_value, 'd-MMM-yyyy')
  WHEN column_value RLIKE '-A(br|pr)-'THEN TO_DATE(REPLACE(column_value, 'Abr', 'Apr'), 'd-MMM-yyyy')
  WHEN column_value RLIKE '-M(ai|ay)-' THEN TO_DATE(REPLACE(column_value, 'Mai', 'May'), 'd-MMM-yyyy')
  WHEN column_value RLIKE '-Jun-' THEN TO_DATE(column_value, 'd-MMM-yyyy')                        
  WHEN column_value RLIKE '-Jul-' THEN TO_DATE(column_value, 'd-MMM-yyyy')
  WHEN column_value RLIKE '-A(go|ug)-' THEN TO_DATE(REPLACE(column_value, 'Ago', 'Aug'), 'd-MMM-yyyy')
  WHEN column_value RLIKE '-S(et|pt)-' THEN TO_DATE(REPLACE(column_value, 'Set', 'Sep'), 'd-MMM-yyyy')
  WHEN column_value RLIKE '-O(ut|ct)-' THEN TO_DATE(REPLACE(column_value, 'Out', 'Oct'), 'd-MMM-yyyy')
  WHEN column_value RLIKE '-Nov-' THEN TO_DATE(column_value, 'd-MMM-yyyy')
  WHEN column_value RLIKE '-D(ec|ez)-' THEN TO_DATE(REPLACE(column_value, 'Dez', 'Dec'), 'd-MMM-yyyy')

  WHEN column_value RLIKE '^\\d{4}-\\d{1,2}-\\d{1,2}$' THEN TO_DATE(column_value, 'yyyy-M-d')
  WHEN column_value RLIKE '^\\d{1,2}-\\d{1,2}-\\d{4}$' THEN TO_DATE(column_value, 'd-M-yyyy')
	
  ELSE try_cast(column_value AS DATE)
END;

In [0]:
%sql
CREATE OR REPLACE FUNCTION clean_for_timestamp_type(column_value string) RETURNS TIMESTAMP

RETURN CASE
  WHEN column_value IS NULL THEN NULL
  WHEN regexp_replace(column_value, '[€%\\p{Z}]', '') IN ('', 'null', 'NaN') --First, remove every space/separator in Unicode, in addition to % and €
    OR column_value RLIKE '^[€%,.\\p{Z}]+$' THEN NULL --It consists of '[€%,.\\p{Z}]' and nothing else
  
--'d' accepts 1 or 2 digits for days
  WHEN column_value RLIKE '-Jan-' THEN to_timestamp(column_value, 'd-MMM-yyyy H:m')
  --If it's already "Fev", REPLACE does nothing
  WHEN column_value RLIKE '-F(eb|ev)-' THEN to_timestamp(REPLACE(column_value, 'Fev', 'Feb'), 'd-MMM-yyyy H:m')
  WHEN column_value RLIKE '-Mar-' THEN to_timestamp(column_value, 'd-MMM-yyyy H:m')
  WHEN column_value RLIKE '-A(br|pr)-'THEN to_timestamp(REPLACE(column_value, 'Abr', 'Apr'), 'd-MMM-yyyy H:m')
  WHEN column_value RLIKE '-M(ai|ay)-' THEN to_timestamp(REPLACE(column_value, 'Mai', 'May'), 'd-MMM-yyyy H:m')
  WHEN column_value RLIKE '-Jun-' THEN to_timestamp(column_value, 'd-MMM-yyyy H:m')                        
  WHEN column_value RLIKE '-Jul-' THEN to_timestamp(column_value, 'd-MMM-yyyy H:m')
  WHEN column_value RLIKE '-A(go|ug)-' THEN to_timestamp(REPLACE(column_value, 'Ago', 'Aug'), 'd-MMM-yyyy H:m')
  WHEN column_value RLIKE '-S(et|pt)-' THEN to_timestamp(REPLACE(column_value, 'Set', 'Sep'), 'd-MMM-yyyy H:m')
  WHEN column_value RLIKE '-O(ut|ct)-' THEN to_timestamp(REPLACE(column_value, 'Out', 'Oct'), 'd-MMM-yyyy H:m')
  WHEN column_value RLIKE '-Nov-' THEN to_timestamp(column_value, 'd-MMM-yyyy H:m')
  WHEN column_value RLIKE '-D(ec|ez)-' THEN to_timestamp(REPLACE(column_value, 'Dez', 'Dec'), 'd-MMM-yyyy H:m')

  WHEN column_value RLIKE '^\\d{4}-\\d{1,2}-\\d{1,2} \\d{2}:\\d{2}$' THEN to_timestamp(column_value, 'yyyy-M-d H:m') 
	WHEN column_value RLIKE '^\\d{1,2}-\\d{1,2}-\\d{4} \\d{2}:\\d{2}$' THEN to_timestamp(column_value, 'd-M-yyyy H:m')

  ELSE try_to_timestamp(column_value) 
END;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.bd_rede_hyundai 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.bd_rede_hyundai;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.campaigns 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.campaigns;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.campanha_vouchers
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.campanha_vouchers; 

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.campanhas_tecnicas 
SELECT  *  EXCEPT(_fivetran_id, 
                  created_time, 
                  modified_time,
                  dist_claim_date,
                  dlr_claim_date,
                  last_activity_time,
                  data_reparacao,
                  unsubscribed_time,
                  warranty_start_date,
                  _fivetran_deleted,
                  _fivetran_synced),

        cast(_fivetran_id as bigint) as _fivetran_id,
        
        -- Dates
        cast(clean_for_timestamp_type(created_time) AS TIMESTAMP) AS created_time,
        cast(clean_for_timestamp_type(modified_time) AS TIMESTAMP) AS modified_time,
        cast(clean_for_date_type(dist_claim_date) AS DATE) AS dist_claim_date,
        cast(clean_for_date_type(dlr_claim_date) AS DATE) AS dlr_claim_date,
        cast(clean_for_timestamp_type(last_activity_time) AS TIMESTAMP) AS last_activity_time,
        cast(clean_for_date_type(data_reparacao) AS DATE) AS data_reparacao,
        cast(clean_for_timestamp_type(unsubscribed_time) AS TIMESTAMP) AS unsubscribed_time,
        cast(clean_for_date_type(warranty_start_date) AS DATE) AS warranty_start_date,
       
       -- Fivetran metadata
       cast(_fivetran_deleted as boolean) as _fivetran_deleted,
       cast(_fivetran_synced as timestamp) as _fivetran_synced
 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.campanhas_tecnicas;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.contactos_importador_pbs 
SELECT cast(_fivetran_id as bigint) as _fivetran_id,
       
       cast(clean_for_timestamp_type(created_time) AS TIMESTAMP) AS created_time,
       cast(origem as string) as origem,
       cast(nome_proprio as string) as nome_proprio,
       cast(modelo as string) as modelo,
       cast(tipo_cliente as string) as tipo_cliente,
       cast(id_contactos as string) as id_contactos,
       cast(cliente_hyundai as string) as cliente_hyundai,
       cast(tipo_viatura as string) as tipo_viatura,
       cast(id as bigint) as id,
       cast(clean_for_timestamp_type(processed_at) AS TIMESTAMP) AS processed_at,
      
      -- Fivetran metadata
	cast(_fivetran_deleted as boolean) as _fivetran_deleted,
	cast(_fivetran_synced as timestamp) as _fivetran_synced

  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.contactos_importador_pbs;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.contactos_pbs 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.contactos_pbs;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.contas_importador_pbs 
SELECT cast(_fivetran_id as bigint) as _fivetran_id,
       
       cast(id as bigint) as id,
       cast(clean_for_timestamp_type(created_time) AS TIMESTAMP) AS created_time,
       cast(origem as string) as origem,
       cast(id_contas as string) as id_contas,
       cast(cliente_hyundai as string) as cliente_hyundai,
       
       -- Fivetran metadata
       cast(_fivetran_deleted as boolean) as _fivetran_deleted,
		   cast(_fivetran_synced as timestamp) as _fivetran_synced
 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.contas_importador_pbs;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.contas_pbs 
SELECT *
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.contas_pbs;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.contratos_financiamento 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.contratos_financiamento;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.deals
SELECT 
cast(_fivetran_id as bigint) as _fivetran_id,
cast(deal_owner_name as string) as deal_owner_name,
cast(id_contacto as bigint) as id_contacto,
cast(clean_for_timestamp_type(created_time) AS TIMESTAMP) AS created_time,
cast(clean_for_timestamp_type(closing_date) AS TIMESTAMP) AS closing_date,
cast(deal_owner as bigint) as deal_owner,
cast(stage as string) as stage,
cast(forecast_type as string) as forecast_type,
cast(conta_name as bigint) as conta_name,
cast(age_in_days as int) as age_in_days,
cast(age_tier as string) as age_tier,
cast(tasks_involved as string) as tasks_involved,
cast(events_involved as string) as events_involved,
cast(calls_involved as string) as calls_involved,
cast(activities_involved as string) as activities_involved,
CAST(tasks_only AS BOOLEAN) as tasks_only,
CAST(events_only AS BOOLEAN) as events_only,
CAST(calls_only AS BOOLEAN) as calls_only,
cast(origem_do_negocio as string) as origem_do_negocio,
cast(contacto_name as string) as contacto_name,
cast(modelos as string) as modelos,
cast(qtd_viaturas as int) as qtd_viaturas,
cast(lead_conversion_time AS INT) AS lead_conversion_time,
cast(sales_cycle_duration as int) as sales_cycle_duration,
cast(overall_sales_duration as int) as overall_sales_duration,
cast(tipo_de_negocio as string) as tipo_de_negocio,
cast(motivo_da_perda as string) as motivo_da_perda,
cast(clean_for_date_type(data_prevista_de_entrega) AS DATE) AS data_prevista_de_entrega,
cast(clean_for_date_type(data_prevista_matricula) AS DATE) AS data_prevista_matricula,
cast(num_de_propostas as int) as num_de_propostas,
cast(id_negocio as bigint) as id_negocio,
cast(clean_for_date_type(data_prevista_de_reentrada_em_negocio) AS DATE) AS data_prevista_de_reentrada_em_negocio,
cast(id_conta as bigint) as id_conta,
cast(codigo_negocio as string) as codigo_negocio,
cast(f2 as bigint) as f2,
cast(concessao as string) as concessao,
cast(clean_for_date_type(data_criacao_da_lead) AS DATE) AS data_criacao_da_lead,
cast(formulario as string) as formulario,
cast(instalacao as string) as instalacao,
cast(chave_concessao_bd_rede as string) as chave_concessao_bd_rede,
cast(chave_instalacao_bd_rede as string) as chave_instalacao_bd_rede,
cast(chave_contrato_concessionario_bd_rede as string) as chave_contrato_concessionario_bd_rede,
cast(contrato_concessionario as string) as contrato_concessionario,
cast(clean_for_date_type(data_venda) AS DATE) AS data_venda,
cast(chave_agrupamento_performance_vendas as string) as chave_agrupamento_performance_vendas,
cast(agrupamento_performance_vendas as string) as agrupamento_performance_vendas,
cast(chave_agrupamento_cliente as string) as chave_agrupamento_cliente,
cast(agrupamento_cliente as string) as agrupamento_cliente,
cast(caracterizacao as string) as caracterizacao,
cast(nome_contacto as string) as nome_contacto,
cast(n__matriculas_associadas as int) as n__matriculas_associadas,
cast(n__matriculas_por_associar as int) as n__matriculas_por_associar,
cast(modelo_de_venda as string) as modelo_de_venda,
cast(campanha as bigint) as campanha,
cast(modelovendaupdate as string) as modelovendaupdate,
cast(clean_for_decimal_type(probability____) AS DECIMAL(29,2)) AS probability____,
cast(tipo_cliente_negocio as string) as tipo_cliente_negocio,
cast(clean_for_date_type(data_decisao_negocio) AS DATE) AS data_decisao_negocio,
cast(categoria_negocio as string) as categoria_negocio,
cast(categoria_negocio_venda as string) as categoria_negocio_venda,
cast(converted_from_lead as bigint) as converted_from_lead,

-- Fivetran metadata
cast(_fivetran_deleted as boolean) as _fivetran_deleted,
cast(_fivetran_synced as timestamp) as _fivetran_synced
  		 
FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.deals;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.historico_de_servicos 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.historico_de_servicos;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.leads_pbs 
SELECT  cast(_fivetran_id as bigint) as _fivetran_id,
        
        cast(id as bigint) as id,
        cast(lead_owner as bigint) as lead_owner,
        cast(lead_owner_name as string) as lead_owner_name,
        cast(lead_source as string) as lead_source,
        cast(lead_status as string) as lead_status,
        cast(lead_status_2 as string) as lead_status_2,
        cast(clean_for_timestamp_type(created_time) AS TIMESTAMP) AS created_time,
        cast(is_converted as string) as is_converted,
        cast(tasks_involved as string) as tasks_involved,
        cast(events_involved as string) as events_involved,
        cast(calls_involved as string) as calls_involved,
        cast(activities_involved as int) as activities_involved,
        cast(created_by as bigint) as created_by,
        cast(distrito as string) as distrito,
        cast(pais as string) as pais,
        cast(formulario as string) as formulario,
        cast(tipo_de_pedido as string) as tipo_de_pedido,
        cast(modelo as string) as modelo,
        cast(consentimento as string) as consentimento,
        cast(concessao as string) as concessao,
        cast(id_lead as bigint) as id_lead,
        CAST(instalacao AS string) AS instalacao,
        CAST(chave_instalacao_bd_rede AS string) AS chave_instalacao_bd_rede,
        CAST(chave_concessao_bd_rede AS string) AS chave_concessao_bd_rede,
        CAST(contrato_concessionario AS string) AS contrato_concessionario,
        CAST(chave_contrato_concessionario_bd_rede AS string) AS chave_contrato_concessionario_bd_rede,
        CAST(concessionario_escolhido AS string) AS concessionario_escolhido,
        CAST(agrupamento_performance_vendas AS string) AS agrupamento_performance_vendas,
        CAST(chave_agrupamento_performance_vendas AS string) AS chave_agrupamento_performance_vendas,
        CAST(chave_agrupamento_cliente AS string) AS chave_agrupamento_cliente,
        CAST(agrupamento_cliente AS string) AS agrupamento_cliente,
        cast(modified_by as bigint) as modified_by,
        cast(clean_for_date_type(data_de_validade_de_consentimento_hyundai) AS DATE) AS data_de_validade_de_consentimento_hyundai,
        CAST(hubleads__nao_comunica_com_cc_ AS string) AS hubleads__nao_comunica_com_cc_,
        cast(clean_for_decimal_type(lead_response_time) AS DECIMAL(29,2)) AS lead_response_time,
        cast(converted_contact as bigint) as converted_contact,
        cast(converted_deal as bigint) as converted_deal,
        cast(clean_for_timestamp_type(converted_date_time) AS TIMESTAMP) AS converted_date_time,
        
        -- Fivetran metadata
        cast(_fivetran_deleted as boolean) as _fivetran_deleted,
		    cast(_fivetran_synced as timestamp) as _fivetran_synced
 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.leads_pbs;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.propostas_realizadas

	    -- ID columns
SELECT  cast(_fivetran_id as bigint) as _fivetran_id,
	    cast(id as bigint) as id,
	    
		-- Basic info columns (containing Dates with validation)
	  cast(clean_for_timestamp_type(created_time) AS TIMESTAMP) AS created_time,
		cast(subject as string) as subject,
		cast(proposta_realizada_owner as bigint) as proposta_realizada_owner,
		cast(proposta_realizada_owner_name as string) as proposta_realizada_owner_name,
		cast(proposta_realizada_stage as string) as proposta_realizada_stage,
		cast(clean_for_timestamp_type(modified_time) AS TIMESTAMP) AS modified_time,
		
		-- Contact and account info
		cast(conta_name as string) as conta_name,
		cast(id_contacto as bigint) as id_contacto,
		cast(contacto_name as bigint) as contacto_name,
		cast(deal_name as bigint) as deal_name,
		
		-- Financial columns (removing € symbol, handling spaces in negative values, handling European decimal format)
		cast(clean_for_decimal_type(sub_total) AS DECIMAL(29,2)) AS sub_total,
		cast(clean_for_decimal_type(grand_total) AS DECIMAL(29,2)) AS grand_total,
		cast(created_by as bigint) as created_by,
		cast(modified_by as bigint) as modified_by,
		cast(concessao as string) as concessao,
		cast(clean_for_decimal_type(preco_base) AS DECIMAL(29,2)) AS preco_base,
		cast(clean_for_decimal_type(isv) AS DECIMAL(29,2)) AS isv,
		cast(clean_for_decimal_type(pintura) AS DECIMAL(29,2)) AS pintura,
		cast(clean_for_decimal_type(base_tributavel) AS DECIMAL(29,2)) AS base_tributavel,
		cast(clean_for_decimal_type(legalizacao) AS DECIMAL(29,2)) AS legalizacao,
		cast(clean_for_decimal_type(desconto_total) AS DECIMAL(29,2)) AS desconto_total,
		cast(clean_for_decimal_type(iuc) AS DECIMAL(29,2)) AS iuc,
		cast(clean_for_decimal_type(preco_total) AS DECIMAL(29,2)) AS preco_total,
		cast(iva as int) as iva,
		
		-- Vehicle info
		cast(modelo as string) as modelo,
		cast(descricao_apoio as string) as descricao_apoio,
		
		cast(clean_for_decimal_type(apoio_concessionario) as DECIMAL(29,2)) as apoio_concessionario,
		
		cast(clean_for_decimal_type(apoio_hyundai_portugal) AS DECIMAL(29,2)) AS apoio_hyundai_portugal,
		cast(clean_for_decimal_type(apoio_total) AS DECIMAL(29,2)) AS apoio_total,
		cast(clean_for_decimal_type(sub_total_com_extras) AS DECIMAL(29,2)) AS sub_total_com_extras,
		cast(clean_for_decimal_type(valor_extras) AS DECIMAL(29,2)) AS valor_extras,
		cast(clean_for_decimal_type(sub_total_com_desconto) AS DECIMAL(29,2)) AS sub_total_com_desconto,
		cast(clean_for_decimal_type(taxa) AS DECIMAL(29,2)) AS taxa,
		cast(clean_for_decimal_type(total_final) AS DECIMAL(29,2)) AS total_final,
		
		-- Status and proposal info
		cast(estado_do_pedido as string) as estado_do_pedido,
		cast(clean_for_date_type(data_de_criacao_da_proposta) AS DATE) AS data_de_criacao_da_proposta,
		
		cast(s_g_p_u_ as string) as s_g_p_u_,
		cast(codigo_da_proposta as string) as codigo_da_proposta,
		cast(descricao_pintura as string) as descricao_pintura,
		
		cast(clean_for_decimal_type(valor_do_apoio_pedido) AS DECIMAL(29,2)) as valor_do_apoio_pedido,	
		cast(clean_for_decimal_type(pvp) AS DECIMAL(29,2)) as pvp,

		cast(clean_for_date_type(data_de_entrega_da_proposta) AS DATE) AS data_de_entrega_da_proposta,
		cast(id_conta as bigint) as id_conta,
		cast(id_negocio as bigint) as id_negocio,
		cast(id_proposta_realizada as bigint) as id_proposta_realizada,
		
		cast(clean_with_null(data_de_validade_da_proposta) as string) as data_de_validade_da_proposta,

		cast(clean_for_decimal_type(desconto_total__c__apoio_de_importador_) AS DECIMAL(29,2)) as desconto_total__c__apoio_de_importador_,

		-- Vehicle specification codes
		cast(codigo_modelo as string) as codigo_modelo,
		cast(sufixo_modelo as string) as sufixo_modelo,
		cast(codigo_cor_exterior as bigint) as codigo_cor_exterior,
		cast(codigo_cor_interior as bigint) as codigo_cor_interior,
		cast(concessionario_owner as string) as concessionario_owner,
		cast(clean_for_decimal_type(valor_aprovado) AS DECIMAL(29,2)) AS valor_aprovado,
		
		-- Dates
		cast(clean_for_date_type(data_prevista_matricula) AS DATE) AS data_prevista_matricula,
		cast(clean_for_date_type(data_prevista_de_entrega) AS DATE) AS data_prevista_de_entrega,
		cast(clean_for_date_type(data_da_conclusao) AS DATE) AS data_da_conclusao,
		
		-- Installation and keys
		cast(instalacao as string) as instalacao,
		cast(chave_instalacao_bd_rede as string) as chave_instalacao_bd_rede,
		cast(chave_concessao_bd_rede as string) as chave_concessao_bd_rede,
		cast(qtd_viaturas as int) as qtd_viaturas,
		cast(versao as string) as versao,
		cast(cod_proposta as string) as cod_proposta,
		cast(codigo_fabricante_cor_interior as string) as codigo_fabricante_cor_interior,
		cast(codigo_fabricante_cor_exterior as string) as codigo_fabricante_cor_exterior,
		cast(estado_ordering as string) as estado_ordering,
		
		-- Classification info
		cast(id_classe as string) as id_classe,
		cast(descricao_classe as string) as descricao_classe,
		cast(id_model_group as string) as id_model_group,
		cast(descricao_model_group as string) as descricao_model_group,
		cast(chave_agrupamento_cliente as string) as chave_agrupamento_cliente,
		cast(tipo_cliente as string) as tipo_cliente,
		cast(combustivel as string) as combustivel,
		cast(id_combustivel as int) as id_combustivel,
		cast(id_modelo as int) as id_modelo,
		cast(tylacode as string) as tylacode,
		
		-- Campaign and support values
		cast(clean_for_decimal_type(valor_campanha) AS DECIMAL(29,2)) AS valor_campanha,
		cast(clean_for_decimal_type(apoio_campanha_hyundai) AS DECIMAL(29,2)) AS apoio_campanha_hyundai,
		cast(clean_for_decimal_type(apoio_financiamento_cetelem) AS DECIMAL(29,2)) AS apoio_financiamento_cetelem,
		cast(clean_for_decimal_type(apoio_retoma_hyundai) AS DECIMAL(29,2)) AS apoio_retoma_hyundai,
		cast(clean_for_decimal_type(comparticipacao_campanha_da_concessao) AS DECIMAL(29,2)) AS comparticipacao_campanha_da_concessao,
		cast(clean_for_decimal_type(comparticipacao_retoma_concessao) AS DECIMAL(29,2)) AS comparticipacao_retoma_concessao,
		
		-- Additional info
		cast(forma_de_pagamento as string) as forma_de_pagamento,
		cast(categoria_proposta as string) as categoria_proposta,
		cast(gestor_area as string) as gestor_area,
		cast(clean_for_decimal_type(margem_frota_eni) AS DECIMAL(29,2)) AS margem_frota_eni,
		cast(clean_for_decimal_type(valor_base) AS DECIMAL(29,2)) AS valor_base,
		
		-- Fivetran metadata
		cast(_fivetran_index as bigint) as _fivetran_index,
		cast(_fivetran_deleted as boolean) as _fivetran_deleted,
		cast(_fivetran_synced as timestamp) as _fivetran_synced

FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.propostas_realizadas;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.viaturas 
SELECT * FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.viaturas;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.viaturas_demo 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.viaturas_demo;